# Polak Ribiere Gradient Descent

Variation with a slightly different update to gradient descent, convergence is similar though.

In [3]:
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad

In [4]:
#takes a numpy array
def f(x):
    return np.array([np.exp(x[0])+np.exp((-1)*x[0]+4.) + x[1]**2])
    #return (x-7)*(x-3)

In [18]:
def MSE(y,x):
    return np.linalg.norm(y-x)

In [47]:
# general class of linesearch iterative methods for continuous optimisation
class LineSearch:
    
    def __init__(self,direction_func,backtracking,step):
        self.direction = direction_func
        self.backtracking = backtracking
        self.step = step
        
        self.sigma = 0.1
        self.tol = 0.00000001
        self.rho = 1/2
        
    
    def Armijo_test(self,f,x,step,direction):
        a = f(x+step*direction)
        b = f(x) - self.sigma*step*np.dot(direction,direction)
        return (a < b).all()
    
    #starts from previous step and finds suitable step length by testing for the Armijo condition
    def find_step(self,f,x,step,direction):
        iters =0
        
        while not self.Armijo_test(f,x,step,direction):
            step = self.rho * step
            iters +=1
            if iters > 10000:
                return None
        return step
        
    #minimise f from starting point x
    def find_minimum(self,f,x):
        #executes steps until convergence
        prev_x = 10*10*10
        iters = 0
        step = self.step
        
        grad = elementwise_grad(f)(x)
        #set initial direction to steepest descent
        direction = np.negative(grad)
        
        
        while np.linalg.norm(x-prev_x) > self.tol:
            
            prev_grad = grad
            grad = elementwise_grad(f)(x)
            direction = self.direction(f,x,direction,grad,prev_grad)
            
            if self.backtracking :
                step = self.find_step(f,x,step,direction)
            if step == None:
                print("step not found")
                break
            
            prev_x = x
            
            #we use previous direction to compute the next
            
            x = x + step*direction
            
            if iters > 10000:
                print("iteration limit reached")
                return
            else:
                #if iters % 10 == 0:
                print("progress")
                print(x)
                print(f(x))
                print()
                iters +=1
        print("converged in %d iterations to" %iters)
        print("x value :", x)
        print("out : ", f(x))

In [48]:
# ||| x float |||
def polak_ribiere(f,x,prev_dir,g,prev_g):
    conjugate_g = np.dot((g-prev_g).T,g)/np.dot(prev_g,prev_g)
    new_g = np.negative(g) + np.dot(prev_dir,conjugate_g)
    return new_g

In [49]:
polak_ribiere = LineSearch(polak_ribiere,step = 100.,backtracking=True)

In [50]:

start = np.ones(10)
y = np.array([1.,21.,32.,43.,54.,65.,76.,87.,98.,109.])
def MSE_y(x):
    return MSE(y,x)
polak_ribiere.find_minimum(MSE_y,start)

progress
[ 1.         10.52121907 15.75788956 20.99456004 26.23123053 31.46790102
 36.70457151 41.94124199 47.17791248 52.41458297]
110.05713508471928

progress
[  1.          20.04243814  30.51577911  40.98912009  51.46246106
  61.93580204  72.40914301  82.88248399  93.35582496 103.82916594]
10.057135084719286

progress
[  1.          21.23259052  32.36051531  43.48844009  54.61636488
  65.74428966  76.87221445  88.00013924  99.12806402 110.25598881]
2.4428649152807083

step not found
converged in 3 iterations to
x value : [  1.          21.23259052  32.36051531  43.48844009  54.61636488
  65.74428966  76.87221445  88.00013924  99.12806402 110.25598881]
out :  2.4428649152807083


In [62]:
g = elementwise_grad(f)

In [63]:
g([2.,3.])

[array(8.), array(0.)]

In [120]:
MSE(y,start)

array([ 0.,  1.,  4.,  9., 16., 25., 36., 49., 64., 81.])